In [1]:
%run common.ipynb

Tensorflow version: 1.15.0
Keras version: 2.2.4
Numpy version: 1.21.2


Using TensorFlow backend.


In [2]:
from deeplift.layers import NonlinearMxtsMode
import deeplift.conversion.kerasapi_conversion as kc
from collections import OrderedDict
import deeplift
from deeplift.util import compile_func
from tqdm import tqdm

In [3]:
SIX5_disc1 = 'SIX5_disc1'
MYC_disc1 = 'MYC_disc1'
SRF_disc1 = 'SRF_disc1'
AP1_disc1 = 'AP1_disc1'
GATA_disc1 = 'GATA_disc1'
TAL1_known1 = 'TAL1_known1'
IRF_known1 = 'IRF_known1'
all_motifs = [SRF_disc1, AP1_disc1, GATA_disc1, TAL1_known1, IRF_known1]

In [4]:
output = []
data_bundles = []
outputs = []

# all_runs = [('model/model_' + motif + '.h5', 
#              'model/model_' + motif + '.json', 
#              'data/testing_' + motif + '.npy', 
#              check(motif)) 
#             for motif in all_motifs]
all_runs = []
for i in range(len(all_motifs)):
    for j in range(i + 1, len(all_motifs)):
        all_runs.append(('model/model_' + str(i) + '_' + str(j) + '.h5', 
             'model/model_' + str(i) + '_' + str(j) + '.json', 
             'data/testing_' + str(i) + '_' + str(j) + '.npy', 
             check_or(all_motifs[i], all_motifs[j])))


for keras_model_weights, keras_model_json, testing_data_path, method in tqdm(all_runs):
    data = load_data(method, testing_data_path)

    model = kc.convert_model_from_saved_files(
                h5_file=keras_model_weights,
                json_file=keras_model_json,
                nonlinear_mxts_mode=NonlinearMxtsMode.DeepLIFT_GenomicsDefault)
      
    score_func = model.get_target_contribs_func(find_scores_layer_idx=0, target_layer_idx=-2)

    background = OrderedDict([('A', 0.3), ('C', 0.2), ('G', 0.2), ('T', 0.3)])

    all_scores = np.zeros((3, data.onehot.shape[0], data.onehot.shape[1]))
    for task_idx in [0,1,2]:
        scores = np.array(score_func(
                                task_idx=task_idx,
                                input_data_list=[data.onehot],
                                input_references_list=[
                                    np.array([
                                        background['A'], 
                                        background['C'], 
                                        background['G'], 
                                        background['T']])
                                    [None,None,:]],
                                batch_size=200,
                                progress_update=None))
    scores = np.sum(scores, axis=2)
    all_scores[task_idx] = scores
    
    all_scores = np.average(all_scores, axis=0)
    
    motif_scores = OrderedDict()
    for motif in all_motifs:
        motif_scores[motif] = []

    for idx in range(len(scores)):
        for embedding in data.embeddings[idx]:
            for motif in all_motifs:
                if motif in embedding.what.getDescription():
                    motif_scores[motif].append(
                        np.mean(all_scores[idx, embedding.startPos:embedding.startPos+len(embedding.what)]))
    
    for motif in all_motifs:
        motif_scores[motif] = np.array(motif_scores[motif])
print(motif_scores)

  0%|          | 0/10 [00:00<?, ?it/s]

nonlinear_mxts_mode is set to: DeepLIFT_GenomicsDefault



/home/wenboc/anaconda3/envs/IiCB/lib/python3.7/site-packages/deeplift/conversion/kerasapi_conversion.py:366: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  model_weights = h5py.File(h5_file)


For layer 1 the preceding linear layer is 0 of type Conv1D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale
For layer 3 the preceding linear layer is 2 of type Conv1D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale

For layer 7 the preceding linear layer is 6 of type Dense;
In accordance with nonlinear_mxts_modeDeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to RevealCancel
For layer 10 the preceding linear layer is 9 of type Dense;
In accordance with nonlinear_mxts_modeDeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to RevealCancel










 10%|█         | 1/10 [00:04<00:36,  4.04s/it]

...

nonlinear_mxts_mode is set to: DeepLIFT_GenomicsDefault
For layer 1 the preceding linear layer is 0 of type Conv1D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale
For layer 3 the preceding linear layer is 2 of type Conv1D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale
For layer 7 the preceding linear layer is 6 of type Dense;
In accordance with nonlinear_mxts_modeDeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to RevealCancel
For layer 10 the preceding linear layer is 9 of type Dense;
In accordance with nonlinear_mxts_modeDeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to RevealCancel


 90%|█████████ | 9/10 [00:51<00:06,  6.66s/it]

nonlinear_mxts_mode is set to: DeepLIFT_GenomicsDefault
For layer 1 the preceding linear layer is 0 of type Conv1D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale
For layer 3 the preceding linear layer is 2 of type Conv1D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale
For layer 7 the preceding linear layer is 6 of type Dense;
In accordance with nonlinear_mxts_modeDeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to RevealCancel
For layer 10 the preceding linear layer is 9 of type Dense;
In accordance with nonlinear_mxts_modeDeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to RevealCancel


100%|██████████| 10/10 [00:59<00:00,  5.93s/it]

OrderedDict([('SRF_disc1', array([], dtype=float64)), ('AP1_disc1', array([], dtype=float64)), ('GATA_disc1', array([], dtype=float64)), ('TAL1_known1', array([ 0.1126446 ,  0.03212369, -0.02356345,  0.0309315 ,  0.07087015,
        0.06602492,  0.06805107,  0.0590351 ,  0.03267898,  0.07458857,
        0.0184189 ,  0.06528406,  0.01882375,  0.05917369,  0.00042062,
        0.0659684 ,  0.04981347,  0.12434862,  0.02879099,  0.07053896,
       -0.03219402,  0.05093554,  0.0657932 ,  0.09999326,  0.09791413,
        0.01927837,  0.11128529,  0.08381284,  0.00034301, -0.01494849,
        0.02394462,  0.03145078,  0.04232392, -0.00412812,  0.03613966,
        0.11404991,  0.048266  ,  0.06139076,  0.03093439,  0.02412246,
       -0.00342872,  0.08222659, -0.0055892 ,  0.08065581,  0.03177132,
        0.04374454,  0.05833913,  0.03743068,  0.05780909,  0.10385512,
        0.01899314,  0.03477437,  0.03142899,  0.03874855,  0.06682907,
        0.02764184,  0.00583637,  0.04129617,  0.008280